In [1]:
import pandas as pd
from tqdm import tqdm
import ast
from  transformers import AutoTokenizer
import transformers
import torch

/DATA/atharva_2101cs14/miniconda3/envs/LLMS/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = "mosaicml/mpt-7b-chat"
 
df = pd.read_csv("/DATA/atharva_2101cs14/Negotiation/PPMD Conversastion Data - Final Complete Dataset.csv")

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model)
config = transformers.AutoConfig.from_pretrained(model, trust_remote_code=True)
config.init_device = "cuda:0"
config.max_seq_len = 4096

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device = "cuda:0",
    config=config,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

Instantiating an MPTForCausalLM model from /DATA/atharva_2101cs14/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-chat/64e5c9c9fb53a8e89690c2dee75a5add37f7113e/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.98s/it]


In [4]:
count = 0
lst = []
chat = """"""
questions = []
answers = []
summary = []
trial = 0
for index in range(len(df)):

    torch.cuda.empty_cache()
    count+=1
    print(count)

    if ( index == 11 or index == 31 ):
        continue

    if( df["USER"][index].isnumeric() ):
        if count != 1:
            lst.append(chat)
        chat = """Prompt: You are a electronics sales agent, help the customer make informed decisions about the product, mention the products specifically along with the model. Answer all the questions of the customer as an electronics sales agent."""
    
#         chat = """Prompt: You are a electronics sales agent, help the customer make informed decisions about the product, mention the products specifically along with the model. Answer all the questions of the customer as an electronics sales agent.
# Example-
# Customer: However, the resolution isn't great. Show me the one with better resolution.
# Sales Agent: You should go with Nikon Coolpix 2000 is a digital camera with a resolution of 1632x1024 pixels and a pixel 1 camera that was released in 2002.
# Reasoning: Customer specifically needs a good resolution, the agent provides him with one. Always try to find what customer exactly needs."""

        continue

    else:
        input_ids = tokenizer(chat,  return_tensors="pt")["input_ids"] 
        print(input_ids.size()[1])

        if( input_ids.size()[1] > 800 ):
            trial = 1
            # print("------------------------")
            summarize = """\nSummarize the below conversation in descreptive manner, such that all the important things discussied in the previous conversation are noted.\n"""
            summarize += chat
            summarize += """\n\nSummary: """

            summary_seq = pipeline(
                summarize,
                max_length=4096,
                do_sample=True,
                top_k=1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
                temperature = 0.1
            )

            idx1 = summary_seq[0]["generated_text"].rfind("Summary:")

            chat = summary_seq[0]["generated_text"][idx1:]
            summary.append(chat)
            print(summary_seq[0]["generated_text"])

        chat += """\nCustomer: """
        chat += df["USER"][index]
        questions.append(df["USER"][index])
        chat += """\nSales Agent: """

        sequences = pipeline(
            chat,
            max_length=4096,
            do_sample=True,
            top_k=1,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            temperature = 0.1
        )

        
        chat = sequences[0]["generated_text"]
        idx = chat.rfind("Sales Agent:")
        if(trial):
            print(chat)
            trial = 0

        answers.append( chat[idx+13:] )
    
    if( count == 20 ):
        trial = pd.DataFrame()
        lst.append(chat)
        trial["Ans"] = lst
        trial.to_csv("check_MPT7B.csv")        
        break

/DATA/atharva_2101cs14/miniconda3/envs/LLMS/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1
2
42


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


3
93


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

18
563


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


19
595


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


20
645


In [19]:
summary

['Summary: \nCustomer wants to purchase a new laptop under 2500. He prefers a cheaper i7 CPU and doesn’t want to pay more than 2400. He had a good experience with his old HP laptop and likes the idea of getting another HP laptop. He mentions that he requires a laptop primarily for office work and some occasional media consumption like watching videos. After discussing the two available options, the Dell Inspiron 15 5000 series and the HP Pavilion x360 14-ba010nr, the customer decides to go ahead with the latter because of its versatility and touch screen capability.']

In [20]:
answers

['Sure thing! Based on your requirements, we have two options available - the Dell Inspiron 15 5000 Series (model number 5567) and the HP Pavilion x360 14-ba010nr (model number BA010NR). Both laptops come with similar specifications but differ slightly in terms of design and features. Which one would you be interested in learning more about?',
 '\nBoth models offer an Intel Core i7 processor, so they should meet your requirements. However, the Dell Inspiron 15 5000 Series comes standard with 8GB RAM while the HP Pavilion x360 14-ba010nr offers 16GB RAM as an option. Would you prefer a higher amount of memory?',
 '\nThe HP Pavilion x360 14-ba010nr has received positive reviews from customers who appreciate its sleek design and long battery life. The laptop also includes a touchscreen display which can be useful if you plan on using it for multimedia purposes such as watching movies or playing games. Is this something you might be interested in?',
 "\nBased on your budget, both laptops f

In [22]:
questions

[]

Returning the whole text in pipeline